primaryVertexFilter efficiency

In [1]:
from coffea import hist
import coffea.processor as processor
from coffea.analysis_objects import JaggedCandidateArray
import numpy as np
import matplotlib.pyplot as plt
import awkward
from uproot_methods import TLorentzVectorArray
from FireHydrant.Tools.trigger import Triggers

np.seterr(divide='ignore', invalid='ignore', over='ignore')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
import json
import os
from os.path import join

dataset4mu_ = json.load(open(join(os.getenv('FH_BASE'), 'Notebooks/MC/Samples/signal_4mu_v2.json')))
dataset2mu2e_ = json.load(open(join(os.getenv('FH_BASE'), 'Notebooks/MC/Samples/signal_2mu2e_v2.json')))

In [4]:
class primaryvtxProcessor(processor.ProcessorABC):
    def __init__(self):
        dataset_axis = hist.Cat('dataset', 'dataset')
        cut_axis = hist.Bin('cut', 'passing primaryVertexFilter', 2, 0, 2)
        self._accumulator = processor.dict_accumulator({
            'filterResult': hist.Hist('Events', dataset_axis, cut_axis),
        })
    
    @property
    def accumulator(self):
        return self._accumulator
    
    def process(self, df):
        output = self.accumulator.identity()
        dataset = df['dataset']
        triggermask = np.logical_or.reduce([df[tp] for tp in Triggers])
        
        output['filterResult'].fill(dataset=dataset, cut=df['metfilters_PrimaryVertexFilter'].astype(int))
        return output
    
    def postprocess(self, accumulator):
        return accumulator

- lxy: 0.3 cm

In [6]:
dataset={
    '4mu/mXX-100_mA-5_lxy-0p3': dataset4mu_['mXX-100_mA-5_lxy-0p3'],
    '4mu/mXX-1000_mA-0p25_lxy-0p3': dataset4mu_['mXX-1000_mA-0p25_lxy-0p3'],
    '2mu2e/mXX-100_mA-5_lxy-0p3': dataset2mu2e_['mXX-100_mA-5_lxy-0p3'],
    '2mu2e/mXX-1000_mA-0p25_lxy-0p3': dataset2mu2e_['mXX-1000_mA-0p25_lxy-0p3'],
}

output = processor.run_uproot_job(dataset,
                                  treename='ffNtuplizer/ffNtuple',
                                  processor_instance=primaryvtxProcessor(),
                                  executor=processor.futures_executor,
                                  executor_args=dict(workers=12, flatten=True),
                                  chunksize=500000,
                                 )

Processing: 100%|██████████| 20/20 [00:02<00:00,  9.97items/s]


In [8]:
from collections import defaultdict

res = defaultdict(dict)
h = output['filterResult']
for d in h.identifiers('dataset'):
    res[str(d)]['total'] = h.integrate('dataset', d).integrate('cut').values()[()]
    res[str(d)]['passed']= h.integrate('dataset', d).integrate('cut', slice(1,2)).values()[()]

import pandas as pd
df = pd.DataFrame(res).transpose()
df['eff %'] = df['passed']/df['total']*100
display(df)

,passed,total,eff %
2mu2e/mXX-1000_mA-0p25_lxy-0p3,42163.0,42163.0,100.0
2mu2e/mXX-100_mA-5_lxy-0p3,15599.0,15599.0,100.0
4mu/mXX-1000_mA-0p25_lxy-0p3,43884.0,43884.0,100.0
4mu/mXX-100_mA-5_lxy-0p3,15577.0,15577.0,100.0


- lxy: 300cm

In [9]:
dataset={
    '4mu/mXX-100_mA-5_lxy-300': dataset4mu_['mXX-100_mA-5_lxy-300'],
    '4mu/mXX-1000_mA-0p25_lxy-300': dataset4mu_['mXX-1000_mA-0p25_lxy-300'],
    '2mu2e/mXX-100_mA-5_lxy-300':dataset2mu2e_['mXX-100_mA-5_lxy-300'],
    '2mu2e/mXX-1000_mA-0p25_lxy-300': dataset2mu2e_['mXX-1000_mA-0p25_lxy-300'],
}

output = processor.run_uproot_job(dataset,
                                  treename='ffNtuplizer/ffNtuple',
                                  processor_instance=primaryvtxProcessor(),
                                  executor=processor.futures_executor,
                                  executor_args=dict(workers=12, flatten=True),
                                  chunksize=500000,
                                 )

Processing: 100%|██████████| 20/20 [00:02<00:00,  9.97items/s]


In [10]:
from collections import defaultdict

res = defaultdict(dict)
h = output['filterResult']
for d in h.identifiers('dataset'):
    res[str(d)]['total'] = h.integrate('dataset', d).integrate('cut').values()[()]
    res[str(d)]['passed']= h.integrate('dataset', d).integrate('cut', slice(1,2)).values()[()]

import pandas as pd
df = pd.DataFrame(res).transpose()
df['eff %'] = df['passed']/df['total']*100
display(df)

,passed,total,eff %
2mu2e/mXX-1000_mA-0p25_lxy-300,31099.0,31099.0,100.0
2mu2e/mXX-100_mA-5_lxy-300,10407.0,10407.0,100.0
4mu/mXX-1000_mA-0p25_lxy-300,32828.0,32828.0,100.0
4mu/mXX-100_mA-5_lxy-300,10245.0,10245.0,100.0
